In [9]:
import numpy as np
import matplotlib.pyplot as plt
from mass_spring_system import Mass, Spring, System


In [10]:
# System parameters
mass = Mass(position=[1.0, 0.0], velocity=[0.0, 2.0], mass=1.0)
spring = Spring(k=10.0, rest_length=1.0)
system = System(mass=mass, spring=spring, damping=0.1)

In [11]:
# Simulation

t_max = 20.0
dt = 0.01
positions = system.simulate(t_max, dt)

In [ ]:
# Plot results
plt.figure(figsize=(8, 6))
plt.plot(positions[:, 0], positions[:, 1], label="Trajectory")
plt.title("2D Mass-Spring System")
plt.xlabel("Position X (m)")
plt.ylabel("Position Y (m)")
plt.grid()
plt.legend()
plt.show()

IndentationError: unexpected indent (3252843667.py, line 2)